In [ ]:
'''
The Matlab code has two different uses. One is for the website and the other is to feed the script manually. By setting 
year to 0, you can run this code with a "made up" state. In this case state is made to match the result from read_result
The second case is when you have an actual year and state of interest. That's what I'll be focusing on getting working first
(the case when year != 0).

p1 of the Matlab code is the probability, and it's the main thing that's being calculated
'''

In [1]:
from statistics import mean
from scipy import stats

In [2]:
print("Starting")

Starting


In [5]:
# Here's some dummy data to start building with
#year is any integer (1898-2014) [must be even number]
year = 1988
states = [4,7]
imputeduncontested=0.35
#year_baseline - which year's national result are you comparing against (force to be equal to variable year for now)
year_baseline = year
state_baseline = [1]

In [6]:
print('Parameters: year={year} states={states} yearbaseline={year_baseline}  \
statebaseline={state_baseline}'.format(year=year,states=states,year_baseline=year_baseline, state_baseline=state_baseline))

Parameters: year=1988 states=[4, 7] yearbaseline=1988  statebaseline=[1]


In [7]:
import pandas as pd
def read_results(year, states):
    # Let's read the csv file into a pandas dataframe
    df = pd.read_csv('House_1898_2014_voteshares_notext.csv', header = None)
    # Let's name the columns
    df.columns = ['Year', 'State', 'District', 'D_voteshare', 'Incumbent', 'Winner']
    # We'll only extract those with the correct year
    year_df = df[df['Year'] == year]
    # Now we'll just take the values with the correct states
    result = year_df[year_df['State'].isin(states)]
    # Drop the 'Year' column from the dataframe
    del result['Year']
    # Return the remaining dataframe
    return result

In [8]:
def state_name(state_number):
    statelist = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
                 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NV', 'NH', 'NJ', 'NM',
                 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA',
                 'WA', 'WV', 'WI', 'WY']

    if state_number > 0 and state_number < 51:
        # The input is valid
        # The minus one is to make the IO match Matlab
        return(statelist[state_number - 1])
    else:
        # The input is not valid
        print("Please enter a state number between 1 and 50")

In [9]:
statedata = read_results(year, states)
nationaldata=read_results(yearbaseline,statebaseline)

In [10]:
statedata.head()

,State,District,D_voteshare,Incumbent,Winner
15505,4,1,1.000000,-1000,1
15506,4,2,0.834580,-1000,1
15507,4,3,0.253094,-1000,-1
15508,4,4,0.691942,-1000,1
16814,7,1,0.772443,-1000,1


In [11]:
stateraw = statedata['D_voteshare'] # I'm thinking he's looking for D_voteshare
nationalraw = nationaldata['D_voteshare']

In [12]:
statename = [state_name(state) for state in states]

In [13]:
electionmessage = 'U.S. House election of ' + str(year) + ' in ' + str(statename)

In [14]:
N_delegates=len(stateraw) 
D_districts=stateraw[stateraw>=0.5]
R_districts=stateraw[stateraw<0.5]
N_D=len(D_districts); 
N_R=N_delegates-N_D;

# Let's round stateraw to 5 digits to for the 1.00000's to 1
rounded_stateraw = [round(state,5) for state in stateraw]
anyimputed = 0 in rounded_stateraw or 1 in rounded_stateraw
imputeduncontested=min(imputeduncontested,1)
imputeduncontested=max(imputeduncontested,0)
imputedfloor=min(imputeduncontested,1-imputeduncontested)

stateresults=stateraw
# Turn all the 0's in stateresults into whatever number the imputed floor is
stateresults[stateresults==0] = imputedfloor
stateresults[stateresults==1] = 1-imputedfloor # Setting value on a copy, gives a warning

nationalresults=nationalraw
nationalresults[nationalresults==0] = imputedfloor
nationalresults[nationalresults==1] = 1-imputedfloor

D_mean_raw=mean(stateraw)
R_mean_raw=1-D_mean_raw
D_mean=mean(stateresults)
R_mean=1-D_mean

C:\Users\HMGSYS\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\HMGSYS\Anaconda3\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
if N_D>=2 and N_R>=2:
    [t_stat, p_value] = stats.ttest_ind(D_districts, R_districts)
    print(t_stat)
    print(p_value)

6.86549474686
0.000128948297351
